(acoustics-scattering_discrete)=
## Discrete scatterers

(acoustics-scattering_discrete_why)=
### What are discrete scatterers and why they matter

We call something a "discrete" scatterer when we consider their scattering response by itself. It is extremely useful, because not only can we model individual _things_ in the ocean—a bubble, oil droplet, sand, fish, krill, whale, or even a submarine—as discrete scatterers, we can also model the combined scattering from a group of these things as a _collection_ of discrete scatterers. This allows us to build up models for more complex scenarios, like a patch of seafloor or sea surface, a cloud of bubbles or a school of fish, which we will explore in the [surface](acoustics-scattering_surface) and [volume](acoustics-scattering_volume) scattering sections.

Furthermore, once we know how different types of objects scatterer sound, we can use that knowledge to interpret the echo signals we receive and infer the potential scattering sources. In cases when we already know or are very confident of what the scatterers are, we can even go one step further, to estimate scatterer properties, such as their size, shape, or material composition based on the spectral, temporal, or other features of the echoes.

For simplicity, in this section we will focus on discussing echoes in the backscattering direction in the [monostatic](acoustics-scattering_monostatic_bistatic) scenario.

(acoustics-scattering_discrete_why)=
### Point scatterer

The simplest discrete scatterer model is a "point" scatterer, which is very small and can be thought of as a new sound source that radiates sound after it is excited by the incident waves. This is nice, because we can immdiately apply many concepts we have learned about the [acoustic sources](acoustics-source) here.

We often assume a point scatterer to be _omnidirectional_, meaning that when impinged by sound, it would scatter echoes equally in all directions. This means that regardless of the directions of the source and the receiver, the echo would always be the same.

But of course, in the natural world, most objects scatter sounds differently depending on the direciton of the incident sound and its relative location with respective to the receiver. Therefore, we can model a _directional_ point scatterer, just like how we consider sources with different directional characteristics.


(acoustics-scattering_discrete_regimes)=
### Scattering regimes

As we have seen in the [acoustic sources](acoustics-source) tutorial, when we talk about the _size_ of the scatterer, we are really talking about the _relative size_ of the scatterer compare to the acoustic wavelength. In acoustics, we often use $ka$, a _dimensionless_ number, to quantify this. Here, $k=2\pi/\lambda$ is the acoustic wavenumber, and $a$ is the _characteristic dimension_ of the scatterer, such as the radius of a sphere or the length of a cylinder. $ka=2\pi a/\lambda$ is _dimensionless_, because both $\lambda$ and $a$ are length measures. This allows us to easily compare the echo reponse of a large scatterer at low frquency and a small scatterer at high frequency, not based on the absolute size of the scatterer or the sound frequency, but by the ratio between the wavelength and the scatterer size.

When the acoustic wavelength is very large compared to the scatterer ($ka\ll1$), we call this the "Rayleigh" scattering regime, in which the scattering is dominated by diffraction. On the other extreme, when the acoustic wavelength is small compared to the scatterer ($ka\gg1$), we call this the "geometric" regime, in which the scattering is dominated by reflection. In the Rayleigh regime, the exact shape of the scatterer is often not as critical, and the scattering cross section scales with frequency with a steep slope proportional to $(ka)^4$, whereas in the geometric regime, the scattering cross section often varies around a high-frequency limit. This relative stability of scattering strength across frequency is useful for calibrating instruments. For example, below is the TS spectrum of a solid tungston carbide sphere typically used to calibrate a 38 kHz echosounder system.


ADD FIGURE
TS SPECTRUM FOR CALIBRATION SPHERE SHOWING FREQUENCY DEPENDENCY

(acoustics-scattering_discrete_size_material)=
### Scatterer size and material properties

Scattering phenomena can be complex and depend on the material properties and shape of the object. For example, bubbles or objects that include air in its composition can _resonante_ in the $ka\ll1$ region, producing scattering signals much stronger than similarly sized objects without air. When the object is made of materials very similar to the surrounding water medium, sound can easily transmit and reflect through the object, causing interference that can be observed in the spectrum.

Below you can see the TS spectrum from a sphere of the same size but with different material compositions: dense metal, fluid (jelly-like), and air (bubble):


ADD FIGURE
TS SPECTRUM FOR 3 SPHERE TYPES OF THE SAME SIZE

(acoustics-scattering_discrete_size_materials_widget)=

To get an intuitive feel of how the echoes change depending on the scatterer, try the widget below and see how the TS magnitude and spectral features change depending on the size of a bubble and a fluid sphere.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import spherical_jn, spherical_yn

In [2]:
import matplotlib
matplotlib.use('module://matplotlib_inline.backend_inline')

import ipywidgets as widgets

from IPython.display import display, HTML

In [3]:
def Pn(n: int, x: np.array):
    if np.max(np.abs(x)) > 1:
        raise ValueError("|x| must be smaller than 1!")

    pn = np.zeros((x.size, n + 1))
    pn[:, 0] = 1
    if n >= 1:
        pn[:, 1] = x

    for nn in range(1, n):
        pn[:, nn + 1] = ((2 * nn + 1) * x * pn[:, nn] - nn * pn[:, nn - 1]) / (nn + 1)

    return pn[:, n]

In [4]:
def fluid_fs(scale, out_flag, para):
    DEG2RAD = np.pi / 180
    ns, x0, xe, g, h = para[:5]
    
    # Create ka array
    if scale == 1:
        ka1 = np.linspace(x0, xe, ns)
    else:
        ka1 = np.logspace(np.log10(x0), np.log10(xe), ns)

    # Determine max mode
    if len(para) < 7:
        Nmax = round(np.max(ka1)) + 10
    else:
        Nmax = para[6]

    ka2 = ka1 / h
    m = len(ka1)
    theta = para[5] * DEG2RAD
    x = np.cos(theta)

    n = np.arange(Nmax)
    nl = 2 * n + 1

    # Compute Legendre polynomials Pn(x)
    pn1 = np.array([Pn(ni, x) for ni in n]).squeeze()

    # Bessel functions and derivatives
    jn1 = np.array([spherical_jn(ni, ka1) for ni in n])  # shape (Nmax, m)
    yn1 = np.array([spherical_yn(ni, ka1) for ni in n])
    djn1 = np.array([spherical_jn(ni, ka1, derivative=True) for ni in n])
    dyn1 = np.array([spherical_yn(ni, ka1, derivative=True) for ni in n])

    jn2 = np.array([spherical_jn(ni, ka2) for ni in n])
    djn2 = np.array([spherical_jn(ni, ka2, derivative=True) for ni in n])

    # Avoid division warnings
    with np.errstate(divide='ignore', invalid='ignore'):
        term1 = djn2 * yn1 / (jn2 * djn1) - g * h * dyn1 / djn1
        term2 = djn2 * jn1 / (jn2 * djn1) - g * h
        cn = term1 / term2
        bn = -1 / (1 + 1j * cn)

    # Form function f and amplitude S
    f = np.zeros(m, dtype=complex)
    # S = np.zeros((m, Nmax), dtype=complex)
    for mm in range(m):
        s = nl * pn1 * bn[:, mm]
        f[mm] = np.sum(s)
        # S[mm, :] = s / ka1[mm]

    outx = ka1
    # Choose output based on out_flag
    if out_flag == 1:
        outy = np.abs(2 * f / ka1)  # Modulus of form function
    elif out_flag == 2:
        outy = -1j * 2 * f / ka1  # Complex form function
    elif out_flag == 3:
        outy = np.abs(f) / (np.sqrt(np.pi) * ka1)  # Modulus of normalized scattering amplitude
    elif out_flag == 4:
        outy = -1j * f / (np.sqrt(np.pi) * ka1)  # Complex normalized scattering amplitude
    else:
        raise ValueError("Invalid out_flag value.")

    return outx, outy

In [5]:
# para_fld=[n, x0, xe, g, h, theta_ka]
para_bubble = [1000, 0.001, 25, 0.0012, 0.22, 180]  # bubble at surface
para_fluid = [1000, 0.001, 25, 1.026, 1.017, 180]  # fluid sphere
scale = 2
out_flag = 2  # complex form function


def plot_TS(radius_bubble, radius_fluid, xscale_type):
    freq_bubble = ka_bubble * c / (radius_bubble * 2 * np.pi)
    freq_fluid = ka_fluid * c / (radius_fluid * 2 * np.pi)
    TS_bubble = 20 * np.log10(np.abs(fm_bubble) * radius_bubble/2)
    TS_fluid = 20 * np.log10(np.abs(fm_fluid) * radius_fluid/2)

    plt.figure(figsize=(6, 4))
    plt.plot(freq_bubble, TS_bubble, lw=1.5)
    plt.plot(freq_fluid, TS_fluid, lw=1.5)
    if xscale_type == "log":
        plt.xlim(np.log10(23), 6e3)
        plt.xscale("log")
    else:
        plt.xlim(0, 6e3)
    plt.ylim(-100, 40)
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("TS (dB)")
    plt.title("Scattering from a bubble and a fluid sphere")
    plt.show()


rr = np.arange(1, 10, 1)/10
radius_bubble_slider = widgets.SelectionSlider(
    options=rr,
    value=0.5,  # default value
    description="Bubble radius (m)",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="400px"),
    continuous_update=True
)

radius_fluid_slider = widgets.SelectionSlider(
    options=rr,
    value=0.5,  # default value
    description="Fluid sphere radius (m)",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="400px"),
    continuous_update=True
)

xscale_radio = widgets.RadioButtons(
    options=["linear", "log"],
    value="log",  # default selected
    description="X-axis scale",
    disabled=False,
    style={'description_width': 'initial'},
)


ka_bubble, fm_bubble = fluid_fs(scale, out_flag, para_bubble)
ka_fluid, fm_fluid = fluid_fs(scale, out_flag, para_fluid)

c = 1500  # medium sound speed [m/s]


interactive_plot = widgets.interactive(
    plot_TS, 
    radius_bubble=radius_bubble_slider, 
    radius_fluid=radius_fluid_slider,
    xscale_type=xscale_radio)
display(interactive_plot)

interactive(children=(SelectionSlider(description='Bubble radius (m)', index=4, layout=Layout(width='400px'), …

In practice, how do we apply these concepts to interpret echo data? Even though simple geometrical shapes like spheres or spheroids are rare in nature, they are often pretty good approximations to many discrete scatterers in the ocean. For example, we can model the fish swimbladder as an air-filled sphere or spheroid, and model a krill as an fluid spheroid or cylinder. By observing how the echo spectrum changes across frequency, we can make inference about the type of animals that produced the echoes.

```{Tip}
:class: tip
Check out the [](acoustics-scattering_inference) section to try interpreting an ["echogram!"](acoustics-scattering_echogram)
```

(acoustics-scattering_discrete_orientation)=
### Orientation dependence

What if we actually consider the non-spherical nature of most objects in nature? In that case, the echoes would indeed vary depending on the direction sound impinges on the scatterer. This is especailly when $ka$ is high and the wavelength is relatively short compared to the scatterer size, such that sound scattered from different part of the scatterer would induce stronger constructive and destructive interference patterns in different directions. This is the same concept as discussed in the [acoustic sources](acoustics-source) section, where higher $ka$ means a more directional beampattern.

To get an idea of how this directionality may look like, try the widget below and see how the TS magnitude and spectral features of a fluid spheroid change depending on the incident sound direction.

In [6]:
def dwba_prolate_spheroid(L, a, g, h, ka, phi):
    """
    DWBA solution for a fluid prolate spheroid.
    
    Parameters:
        L   : Length of spheroid (long axis)
        a   : Radius of spheroid (short axis)
        g   : rho2/rho1
        h   : c2/c1
        ka  : dimensionless wavenumber * radius (can be array)
        phi : incident angle in radians

    Returns:
        fbs      : complex backscattering amplitude
        sigmabs  : backscattering cross-section (abs squared)
        TS       : target strength in dB
    """

    Cb = 1 / (g * h**2) + 1 / g - 2  # gamma_kappa - gamma_rho

    beta = phi + np.pi / 2
    ka = np.atleast_1d(ka)  # ensure ka is array

    # Argument inside spherical Bessel function
    sin2 = np.sin(beta)**2
    cos2 = np.cos(beta)**2
    ellip_term = np.sqrt(sin2 + ((L / (2 * a))**2) * cos2)
    arg = 2 * ka / h * ellip_term

    j1 = spherical_jn(1, arg)

    fbs = (ka**2) * L * Cb / 2 * j1 / arg

    return fbs

In [7]:
a = 0.015
L = a * 6
g = 1.043
h = 1.053
c = 1500

freq = np.arange(100, 100e3, 10)
ka = 2*np.pi*freq / c * a

def update_dwba(angle, xscale_type):

    phi = np.deg2rad(angle)  # incident angle in radians (e.g., backscatter)
    fbs = dwba_prolate_spheroid(L, a, g, h, ka, phi)
    TS = 20*np.log10(np.abs(fbs))

    plt.figure(figsize=(5,3.5))
    plt.plot(freq/1e3, TS, lw=1)
    if xscale_type == "log":
        plt.xscale("log")
    plt.xlim(500/1e3, 100e3/1e3)
    plt.ylim(-120, -50)
    plt.xlabel("Frequency (kHz)")
    plt.ylabel("TS (dB)")
    plt.title("Scattering from a fluid prolate spheroid")
    plt.show


# rr = np.hstack(((np.arange(5, 10, 1)/100), np.arange(1, 10, 1)/10))
aa = np.arange(0, 91, 1)
angle_slider = widgets.SelectionSlider(
    options=aa,
    value=0,  # default value
    description="Incident angle (deg)",
    style={'description_width': 'initial'},
    continuous_update=True
)

xscale_radio = widgets.RadioButtons(
    options=["linear", "log"],
    value="linear",  # default selected
    description="X-axis type",
    disabled=False,
    style={'description_width': 'initial'}
)

interactive_plot = widgets.interactive(
    update_dwba, angle=angle_slider, xscale_type=xscale_radio)
display(interactive_plot)

interactive(children=(SelectionSlider(description='Incident angle (deg)', options=(np.int64(0), np.int64(1), n…

### Inferring scatterer identity

Seeing how the TS spectrum can change with both material properties, shape, and incident angle even for simple scatterers such as spheres and spheroid, it is natural to imagine that we can use these features to infer the dominant scattering mechanisms and scatterer identity. "Acoustic color"—the strength of echo as a function of angle and frequency as shown in the fig below—is one such feature that has been used widely to distinguish different types of scatterer underwater.

In [8]:
# ADD ACOUSITC COLOR FIGURE!